<img width="200" style="float:left" 
     src="https://upload.wikimedia.org/wikipedia/commons/f/f3/Apache_Spark_logo.svg" />

# Sections
* [Description](#0)
* [1. Setup](#1)
  * [1.1 Start Hadoop](#1.1)  
  * [1.2 Search for Spark Installation](#1.2)
  * [1.3 Create SparkSession](#1.3)
* [2. Practice](#2)
* [3. TearDown](#3)
  * [3.1 Stop Hadoop](#3.1)

<a id='0'></a>
## Description
<p>
<div>The goal for this notebok is getting familiar with the core concepts of Apache Spark:</div>
<ul>    
    <li>SparkSession</li>
    <li>DataFrames</li>
    <li>RDD</li>
    <li>Partitions</li>
    <li>Actions</li>
    <li>Transformations</li>
</ul>    
</p>

<a id='1'></a>
## 1. Setup

Since we are going to process data stored from HDFS let's start the service

<a id='1.1'></a>
### 1.1 Start Hadoop

Start Hadoop

Open a terminal and execute
```sh
hadoop-start.sh
```

<a id='1.2'></a>
### 1.2 Search for Spark Installation 
This step is required just because we are working in the course environment.

In [ ]:
import findspark
findspark.init()

I'm changing pandas max column width property to improve data displaying

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)

<a id='1.3'></a>
### 1.3 Create SparkSession
By setting this environment variable we can include extra libraries in our Spark cluster

In [ ]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = ' pyspark-shell'

The first thing always is to create the SparkSession

In [ ]:
from pyspark.sql.session import SparkSession

spark = (SparkSession.builder
.appName("Pokemon - Data - Lab")
.config("spark.sql.warehouse.dir","hdfs://localhost:9000/warehouse")
.getOrCreate())

<a id='2'></a>
## 2. Practice

spark variable is an instance of SparkSession class

In [ ]:
spark

we can check the type of a variable just using the type function

In [ ]:
type(spark)

### High Level API

DataFrames is the high level data structure in Apache Spark.

We can create DataFrames in two ways. 

- programmatically
- external data source

In this case we are going to create a DataFrame by "reading" directory in HDFS.

One important thing to remember is that we should **ALWAYS keep the same data format** (csv, json, parquet, ...) for all the files inside a folder.


In [ ]:
pokemon_raw = (spark.read                    
                    .option("header", "true")
                    .csv("hdfs://localhost:9000/datalake/raw/pokemon/pokemon-data/"))

pokemon_raw is an instance of DataFrame class

In [ ]:
type(pokemon_raw)

By printing the variable we can see the column names and their types

In [ ]:
pokemon_raw

let's check its structure (schema)

In [ ]:
pokemon_raw.printSchema()

Spark did not perform any kind of processing up to this point. We are going to call **show** function in the DataFrame, which is an **action**, to ask Spark to show the first 5 elements in the DataFrame. By calling this function, the DataFrame gets triggered and Spark will rewind to the very begining and start to do some work. In this case, the DataFrame has no transformations yet, so is going to ask the first executor to return to the driver the first 5 elements. Once received the driver will print the data in the notebook console

**NOTE**

**show** is an action

In [ ]:
pokemon_raw.show(5,False)

It seems that some columns are numerical not a string 🤔.

We can force Spark to infer the data types of every column when reading CSV files. In this case lets use optional property inferSchema.

**NOTE**

Infering the schema has some penalty in performace since Spark will have to read all the files to actually infer the proper type per column (if there are many files).

In [ ]:
# inferSchema is telling Spark to find out the proper data type of each column
pokemon_raw = (spark.read
              .option("header","true")              
              .option("inferSchema","true")
              .csv("hdfs://localhost:9000/datalake/raw/pokemon/pokemon-data/"))

Now numerical columns are integers.

Is very important to **ALWAYS check we have the proper data types** in each columns, as depending on the data type, we would use different funtions. Spark bundles functions for dealing with numbers, string, dates , ...

In [ ]:
pokemon_raw.printSchema()

In [ ]:
pokemon_raw.show(5,False)

Sometimes is usefull to transform a **Spark DataFrame** into a **Pandas DataFrame** for ploting the data or to use another library that relies/depends on Pandas.

Transforming a **Spark DataFrame** into a **Pandas DataFrame** is a risky operation, since most of the times, we will be working with datasets that can't fit one single machine memory; and if we transform a big **Spark DataFrame** the driver will fail and the application will stop

To take less risks, I'm going to truncate the DataFrame to just 10 elements, and then transform it to Pandas

**NOTE:**

**limit** is a tranformation

**toPandas** is an action

In [ ]:
pokemon_raw.limit(10).toPandas()

### Low Level API

High Level API data structures like DataFrames rely internally in the low level API data structure called RDD.

We can access the interall RDD via the rdd property in the DataFrame variable

In [ ]:
type(pokemon_raw.rdd)

By printing the RDD is not so obvious what holds inside

In [ ]:
pokemon_raw.rdd

All the data, no matter if we are using DataFrames or RDD, is divided in chuncks called **partitions** that are distributed among the available executors.

We can get the number of partitions by calling **getNumPartitions** function in an rdd

In [ ]:
pokemon_raw.rdd.getNumPartitions()

RDD have similar functions to DataFrame ones.

**NOTE**

**take** is an action

In [ ]:
pokemon_raw.rdd.take(10)

We can create RDD directly by using the old SparkContext API

In [ ]:
pokemon_raw = spark.sparkContext.textFile("hdfs://localhost:9000/datalake/raw/pokemon/pokemon-data/")

In [ ]:
type(pokemon_raw)

In the previous example, since we created the rdd from a DataFrame, Spark alredy splitted and casted the values to their proper data types.

Let's check the contents of an RDD, read from the same directory but this time using the RDD API

Well, we have the lines of the file

In [ ]:
pokemon_raw.take(10)

To structure the line in columns we need to take care of spliting the columns using the delimiter ,

In [ ]:
pokemon_raw.map(lambda l: l.split(",")).take(10)

We will still have to cast some columns to integer ...

Working with DataFrames makes possible to just focus on the processing and analytics we need to perform instead of worrying about low-level details like having to deal with the file formats for example.

<a id='3'></a>
## 3. Tear Down

Once we complete the the lab we can stop all the services

<a id='3.1'></a>
### 3.1 Stop Hadoop

Stops Hadoop
Open a terminal and execute
```sh
hadoop-stop.sh
```